In [45]:
import os
os.chdir("/Users/nickl/Documents/College/DATA 481 - Gene Data")

In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Use RandomForestRegressor for regression
from sklearn.metrics import accuracy_score, classification_report

Filtered Gene Expression Dataset Random Forest

In [47]:
#Loading filtered dataset
df = pd.read_csv("filtered_gene_expression.csv")
df.head()

,Unnamed: 0,ENSG00000110484,ENSG00000173467,ENSG00000160180,ENSG00000106541,ENSG00000086548,ENSG00000186081,ENSG00000164128,ENSG00000082175,ENSG00000012223,...,ENSG00000183579,ENSG00000111640,ENSG00000141582,ENSG00000083444,ENSG00000077238,ENSG00000131378,ENSG00000162627,ENSG00000172428,ENSG00000090238,ENSG00000121310
0,TCGA-B6-A0IG-01A-11R-A034-07,16.604568,13.223057,16.503468,16.406028,16.432548,12.923043,6.321402,10.431743,12.610098,...,9.498140,16.407849,11.891926,13.235186,12.394746,9.942146,8.553333,11.502764,10.531759,10.892306
1,TCGA-BH-A0HQ-01A-11R-A034-07,17.999684,12.796037,15.401028,15.135256,10.963789,13.592751,9.657230,13.523402,13.761161,...,9.069928,15.914698,11.445023,13.502874,12.193215,11.164268,10.274965,10.937427,11.517999,12.172644
2,TCGA-BH-A18G-01A-11R-A12D-07,11.449702,3.660178,0.000000,5.166983,10.137522,10.144959,14.228891,5.449170,18.585784,...,9.609232,18.279752,10.998696,15.340948,12.148326,10.899602,10.838234,11.463646,12.285856,10.922859
3,TCGA-A8-A09K-01A-11R-A00Z-07,1.675843,13.126475,10.825054,12.800465,4.375425,7.437230,11.509252,9.890669,9.995210,...,12.416744,15.469124,13.226838,11.836395,9.202800,9.810061,10.297798,11.573526,12.778097,12.220035
4,TCGA-BH-A0BZ-01A-31R-A12P-07,8.222050,11.830127,12.560654,14.547573,9.016715,11.662777,9.213446,7.501901,7.631275,...,9.499313,16.135190,12.777781,13.686927,12.581818,11.524712,9.752092,10.692426,11.690135,10.532674


In [48]:
#loading meta dataset
metadf = pd.read_csv("metadata.csv")
metadf.head() 

,mRNA,PAM50,TumorPurity,ProliferationScore,ER,PR,HER2
0,TCGA-B6-A0IG-01A-11R-A034-07,LumA,0.73,-0.568397,Positive,Positive,Positive
1,TCGA-BH-A0HQ-01A-11R-A034-07,LumA,0.52,-0.821974,NaN,NaN,Negative
2,TCGA-BH-A18G-01A-11R-A12D-07,Basal,0.79,0.423067,Negative,Negative,Negative
3,TCGA-A8-A09K-01A-11R-A00Z-07,LumA,1.00,0.026868,Positive,Positive,Negative
4,TCGA-BH-A0BZ-01A-31R-A12P-07,LumB,0.37,0.247682,Positive,Positive,Negative


In [49]:
#merging meta df with gene expression df
df.rename(columns={'Unnamed: 0': 'mRNA'}, inplace=True)
df = pd.merge(df, metadf, on =  "mRNA", how='left')
df.set_index("mRNA", inplace=True)
df.head()

,ENSG00000110484,ENSG00000173467,ENSG00000160180,ENSG00000106541,ENSG00000086548,ENSG00000186081,ENSG00000164128,ENSG00000082175,ENSG00000012223,ENSG00000151892,...,ENSG00000162627,ENSG00000172428,ENSG00000090238,ENSG00000121310,PAM50,TumorPurity,ProliferationScore,ER,PR,HER2
mRNA,,,,,,,,,,,,,,,,,,,,,
TCGA-B6-A0IG-01A-11R-A034-07,16.604568,13.223057,16.503468,16.406028,16.432548,12.923043,6.321402,10.431743,12.610098,15.059847,...,8.553333,11.502764,10.531759,10.892306,LumA,0.73,-0.568397,Positive,Positive,Positive
TCGA-BH-A0HQ-01A-11R-A034-07,17.999684,12.796037,15.401028,15.135256,10.963789,13.592751,9.657230,13.523402,13.761161,15.761459,...,10.274965,10.937427,11.517999,12.172644,LumA,0.52,-0.821974,NaN,NaN,Negative
TCGA-BH-A18G-01A-11R-A12D-07,11.449702,3.660178,0.000000,5.166983,10.137522,10.144959,14.228891,5.449170,18.585784,8.618219,...,10.838234,11.463646,12.285856,10.922859,Basal,0.79,0.423067,Negative,Negative,Negative
TCGA-A8-A09K-01A-11R-A00Z-07,1.675843,13.126475,10.825054,12.800465,4.375425,7.437230,11.509252,9.890669,9.995210,16.164904,...,10.297798,11.573526,12.778097,12.220035,LumA,1.00,0.026868,Positive,Positive,Negative
TCGA-BH-A0BZ-01A-31R-A12P-07,8.222050,11.830127,12.560654,14.547573,9.016715,11.662777,9.213446,7.501901,7.631275,14.109405,...,9.752092,10.692426,11.690135,10.532674,LumB,0.37,0.247682,Positive,Positive,Negative


In [50]:
#setting up x and y for random forest
df = df.dropna(subset=["ER"])

X = df.drop(columns=["PAM50", "TumorPurity", "ProliferationScore", "ER", "PR", "HER2"]) 
y = df["ER"] 

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
#fitting random forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators: Number of trees

# Train the model
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [52]:
#Creating predictions from the model
y_pred = rf.predict(X_test)

In [53]:
#checking accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy: 0.95
              precision    recall  f1-score   support

    Negative       1.00      0.82      0.90        38
    Positive       0.93      1.00      0.97        97

    accuracy                           0.95       135
   macro avg       0.97      0.91      0.93       135
weighted avg       0.95      0.95      0.95       135



Cross-Validation Filtered Gene Expression Random Forest

In [54]:
from sklearn.model_selection import cross_val_score

In [55]:
#creating model and fitting it using cross validation
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='accuracy')

# Print results
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores):.4f}")
print(f"Standard Deviation: {np.std(cv_scores):.4f}")

Cross-validation scores: [0.94814815 0.93333333 0.94029851 0.91791045 0.90298507]
Mean Accuracy: 0.9285
Standard Deviation: 0.0162


Random Forest on All Protein Coding Genes

In [56]:
pcdf = pd.read_csv("All_Protein_Coding_Genes.csv", low_memory = False)

In [57]:
#pcdf.head()

In [58]:
#merging meta df with gene expression df
pcdf.rename(columns={'Unnamed: 0': 'mRNA'}, inplace=True)
pcdf = pd.merge(pcdf, metadf, on =  "mRNA", how='left')
pcdf.set_index("mRNA", inplace=True)

In [66]:
#setting up x and y for random forest
fpcdf = pcdf.dropna(subset=["ER"])

X = fpcdf.drop(columns=["PAM50", "TumorPurity", "ProliferationScore", "ER", "PR", "HER2"]) 
y = fpcdf["ER"] 

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
#fitting random forest model
pcrf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators: Number of trees

# Train the model
pcrf.fit(X_train, y_train)

y_pred = pcrf.predict(X_test)

#checking accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy: 0.96
              precision    recall  f1-score   support

    Negative       1.00      0.84      0.91        38
    Positive       0.94      1.00      0.97        97

    accuracy                           0.96       135
   macro avg       0.97      0.92      0.94       135
weighted avg       0.96      0.96      0.95       135



Cross Validation RF on All Protein Coding Genes

In [61]:
#creating model and fitting it using cross validation
pcrf = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(pcrf, X, y, cv=5, scoring='accuracy')

# Print results
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores):.4f}")
print(f"Standard Deviation: {np.std(cv_scores):.4f}")

Cross-validation scores: [0.94074074 0.93333333 0.91791045 0.90298507 0.86567164]
Mean Accuracy: 0.9121
Standard Deviation: 0.0266


In [69]:

#checking to make sure a class imbalance doesn't have an effect
class_counts = y.value_counts()
print(class_counts)
510/(510 + 162)

ER
Positive    510
Negative    162
Name: count, dtype: int64


0.7589285714285714